In [1]:
import numpy as np
import torch

from miscellaneous.tiny_sim_wrapper import TinySimWrapper

/home/yanisf/.pyenv/versions/3.12.5/envs/comma-3.12.5/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yanisf/.pyenv/versions/3.12.5/envs/comma-3.12.5/lib/python3.12/site-packages/torchrl/data/replay_buffers/samplers.py:37: UserWarning: Failed to import torchrl C++ binaries. Some modules (eg, prioritized replay buffers) may not work with your installation. This is likely due to a discrepancy between your package version and the PyTorch version. Make sure both are compatible. Usually, torchrl majors follow the pytorch majors within a few days around the release. For instance, TorchRL 0.5 requires PyTorch 2.4.0, and TorchRL 0.6 requires PyTorch 2.5.0.
  warnings.warn(EXTENSION_WARNING)


In [2]:
env = TinySimWrapper(model_path="../models/tinyphysics.onnx", data_directory_path="../data")

In [9]:
env.current_data_path

'../data/17827.csv'

In [8]:
env._reset()

TensorDict(
    fields={
        observation: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([]),
    device=None,
    is_shared=False)

In [24]:
action = env.rand_action()

In [27]:
new_td = env._step(action)

In [28]:
new_td["reward"]

tensor([-22634.0586])